<a href="https://colab.research.google.com/github/quangdad/DataMining/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import missingno as msno

# Load and preprocess data
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/hepatitis/hepatitis.data', header=None, names=['Class', 'Age', 'Sex', 'Steroid', 'Antivirals', 'Fatigue', 'Malaise', 'Anorexia', 'LiverBig', 'LiverFirm', 'SpleenPalpable', 'Spiders', 'Ascites', 'Varices', 'Bilirubin', 'AlkPhosphate', 'SGOT', 'Albumin', 'Protime', 'Histology'])
data = data.replace('?', np.nan).dropna()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('Class', axis=1), data['Class'], test_size=0.2, random_state=42)

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on test set
y_pred = rf_model.predict(X_test)

# Calculate accuracy on test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test set accuracy: {accuracy:.2f}')

# Load and preprocess test data
test_data = pd.read_csv('patients.csv', header=None, names=['Class', 'Age', 'Sex', 'Steroid', 'Antivirals', 'Fatigue', 'Malaise', 'Anorexia', 'LiverBig', 'LiverFirm', 'SpleenPalpable', 'Spiders', 'Ascites', 'Varices', 'Bilirubin', 'AlkPhosphate', 'SGOT', 'Albumin', 'Protime', 'Histology'])
test_data = test_data.replace('?', np.nan).dropna()

# Make predictions on test data
X_test = test_data.drop('Class', axis=1)
y_test = test_data['Class']

#Make predictions on test data
y_pred = rf_model.predict(X_test)

print('Kết quả dự đoán trên tập kiểm tra:')
for i in range(len(y_pred)):
    sample = X_test.iloc[[i]]
    prediction = rf_model.predict(sample)[0]
    if prediction == 1:
        print(f'Mẫu {i+1}: Bệnh nhân không bị nhiễm virus viêm gan B')
    else:
        print(f'Mẫu {i+1}: Bệnh nhân bị nhiễm virus viêm gan B')


Test set accuracy: 0.94
Kết quả dự đoán trên tập kiểm tra:
Mẫu 1: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 2: Bệnh nhân không bị nhiễm virus viêm gan B
Mẫu 3: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 4: Bệnh nhân không bị nhiễm virus viêm gan B
Mẫu 5: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 6: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 7: Bệnh nhân không bị nhiễm virus viêm gan B
Mẫu 8: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 9: Bệnh nhân bị nhiễm virus viêm gan B
Mẫu 10: Bệnh nhân bị nhiễm virus viêm gan B
